In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
# %pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import xgboost as xgb

In [3]:
df = pd.read_csv('train.csv')
dfTest = pd.read_csv('test.csv')
dataset = pd.read_csv('train.csv')

dfe = pd.read_csv('WA_Fn-UseC_-HR-Employee-Attrition.csv')



In [5]:
assert dataset.isnull().sum().sum() == 0

num_col_names = list(dataset.select_dtypes(include='number').columns)
cat_col_names = list(set(dataset.columns) - set(num_col_names))

In [6]:
df = pd.get_dummies(df, columns=cat_col_names)
tensor = torch.from_numpy(df.values)

In [7]:
df.pop("id")

0          0
1          1
2          2
3          3
4          4
        ... 
1672    1672
1673    1673
1674    1674
1675    1675
1676    1676
Name: id, Length: 1677, dtype: int64

In [8]:
labels = df.pop("Attrition")

In [10]:
from torch.utils.data import random_split
from torch.utils.data import TensorDataset
from sklearn.preprocessing import StandardScaler


sca = StandardScaler()
df = sca.fit_transform(df)

tensor = torch.from_numpy(df)
labeltensor = torch.from_numpy(labels.values)

dataset = TensorDataset(tensor, labeltensor)


train_ratio = 0.6 # ratio of data to be used for training

train_len = int(train_ratio * len(dataset))
test_len = len(dataset) - train_len

train_set, test_set = random_split(dataset, [train_len, test_len])


from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(df, labels.values, test_size=0.3)

In [12]:
D_train = xgb.DMatrix(X_train, label=Y_train)
D_test = xgb.DMatrix(X_test, label=Y_test)

In [13]:
param = {
    'eta': 0.3, 
    'max_depth': 3,  
    'objective': 'multi:softprob',  
    'num_class': 3} 

steps = 20  # The number of training iterations

In [14]:
model = xgb.train(param, D_train, steps)

In [15]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, accuracy_score

preds = model.predict(D_test)
best_preds = np.asarray([np.argmax(line) for line in preds])

print("Precision = {}".format(precision_score(Y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(Y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(Y_test, best_preds)))

Precision = 0.7931818181818182
Recall = 0.5945527236381809
Accuracy = 0.878968253968254


In [16]:
ids = dfTest.pop("id")

In [17]:
dfTest = pd.get_dummies(dfTest, columns=cat_col_names)

In [18]:
test_setf = sca.transform(dfTest)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [23]:
final = []
for data in test_setf:
    print(data)
    final.append(model.predict_proba(data))

[ 1.90000000e+01  9.92000000e+02  1.00000000e+00  1.00000000e+00
  1.00000000e+00  4.00000000e+00  4.30000000e+01  3.00000000e+00
  1.00000000e+00  3.00000000e+00  2.31800000e+03  1.77780000e+04
  1.00000000e+00  1.20000000e+01  3.00000000e+00  4.00000000e+00
  8.00000000e+01  3.17774032e-17  1.00000000e+00  2.00000000e+00
  2.00000000e+00  1.00000000e+00  1.48294548e-17  4.02513774e-17
 -6.24955596e-17  1.00000000e+00 -4.02513774e-17  1.00000000e+00
  4.97845984e-17  1.22872626e-16  2.33034290e-17 -4.97845984e-17
  1.00000000e+00 -5.61400790e-17  3.60143903e-17  8.05027548e-17
 -3.60143903e-17  3.17774032e-18  0.00000000e+00  1.19694885e-16
  1.00000000e+00  6.77917935e-17 -2.11849355e-17  4.23698709e-18
  1.00000000e+00 -9.00359758e-17  5.71993258e-17  1.00000000e+00
  7.62657677e-17 -1.27109613e-16  1.00000000e+00 -1.80071952e-17
  1.00000000e+00 -5.53456439e-17]


AttributeError: 'Booster' object has no attribute 'predict_proba'

In [27]:

Output = pd.DataFrame({'id':ids,'Attrition':final})

In [28]:
Output.to_csv('Submision.csv',index=False)
Output.head()

,id,Attrition
0,1677,0.038483
1,1678,0.066631
2,1679,0.000514
3,1680,0.002333
4,1681,0.533811


In [18]:
# Be careful to overwrite our original name file!
model_name = 'simplenet.net'
torch.save(model.state_dict(),model_name)